In [1]:
import pandas as pd
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

%matplotlib inline

In [2]:
works_custs = pd.read_csv('AdvWorksCusts.csv')
works_custs.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(works_custs.columns)

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome'],
      dtype='object')


In [3]:
works_custs.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,4/8/66,Bachelors,Professional,M,M,1,0,0,2,137947
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,5/14/65,Bachelors,Professional,M,S,0,1,3,3,101141
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,8/12/65,Bachelors,Professional,M,M,1,1,3,3,91945
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,2/15/68,Bachelors,Professional,F,S,0,1,0,0,86688
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,8/8/68,Bachelors,Professional,F,S,1,4,5,5,92771


In [4]:
bike_buyer = pd.read_csv('AW_BikeBuyer.csv')
bike_buyer.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(bike_buyer.columns)

Index(['CustomerID', 'BikeBuyer'], dtype='object')


In [5]:
bike_buyer.head()

,CustomerID,BikeBuyer
0,11000,0
1,11001,1
2,11002,0
3,11003,0
4,11004,1


In [6]:
works_custs_bike_buyer = pd.merge(works_custs, bike_buyer, how='left', on='CustomerID')
print(works_custs_bike_buyer.columns)
print(works_custs_bike_buyer)

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'BikeBuyer'],
      dtype='object')
       CustomerID Title   FirstName MiddleName    LastName Suffix  \
0           11000   NaN         Jon          V        Yang    NaN   
1           11001   NaN      Eugene          L       Huang    NaN   
2           11002   NaN       Ruben        NaN      Torres    NaN   
3           11003   NaN     Christy        NaN         Zhu    NaN   
4           11004   NaN   Elizabeth        NaN     Johnson    NaN   
5           11005   NaN       Julio        NaN        Ruiz    NaN   
6           11006   NaN       Janet          G     Alvarez    NaN   
7           11007   NaN    

In [7]:
print(works_custs_bike_buyer.shape)
works_custs_bike_buyer.head()

(16404, 24)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,BikeBuyer
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,Bachelors,Professional,M,M,1,0,0,2,137947,0
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,Bachelors,Professional,M,S,0,1,3,3,101141,1
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,Bachelors,Professional,M,M,1,1,3,3,91945,0
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,Bachelors,Professional,F,S,0,1,0,0,86688,0
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,Bachelors,Professional,F,S,1,4,5,5,92771,1


In [8]:
labels = np.array(works_custs_bike_buyer['BikeBuyer'])
labels.size

16404

In [9]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education', 'Gender', 'MaritalStatus']

Features = encode_string(works_custs_bike_buyer['Occupation'])
for col in categorical_columns:
    temp = encode_string(works_custs_bike_buyer[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)
print(Features[:2, :])    

(16404, 14)
[[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1.]]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [10]:
Features = np.concatenate([Features, np.array(works_custs_bike_buyer[['HomeOwnerFlag','NumberCarsOwned', 
                            'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome']])], axis = 1)
print(Features.shape)
print(Features[:2, :])

(16404, 19)
[[0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00
  1.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 2.00000e+00
  1.37947e+05]
 [0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00
  0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00 3.00000e+00 3.00000e+00
  1.01141e+05]]


In [11]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 1000) #need to have shuffle=False in here for correct answer
X_train = Features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
#X_test = Features[indx[1],:]
#y_test = np.ravel(labels[indx[1]])

In [12]:
y_train

array([0, 1, 0, ..., 1, 1, 0])

In [13]:
from sklearn.feature_selection import SelectKBest

from sklearn.feature_selection import chi2

test = SelectKBest(score_func=chi2, k=4)

In [14]:
fit = test.fit(X_train, y_train)

In [15]:
features = fit.transform(X_train)
np.set_printoptions(precision=3)
print(fit.scores_)

[1.029e+02 2.036e+01 7.914e+01 2.663e+02 5.113e+01 6.870e+01 1.273e+01
 8.898e+00 1.076e+00 5.963e+01 1.098e+02 1.073e+02 2.550e+02 3.015e+02
 6.144e-03 3.669e+02 7.440e+03 1.191e+03 2.555e+07]


In [16]:
X_test1 = Features[indx[1],:]
y_test1 = np.ravel(labels[indx[1]])

In [17]:
scaler = preprocessing.StandardScaler().fit(X_train[:,18:])
X_train[:,18:] = scaler.transform(X_train[:,18:])
X_test1[:,18:] = scaler.transform(X_test1[:,18:])
X_train[:2,]

array([[ 1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         1.   ,  0.   ,  0.   ,  1.   ,  1.   ,  0.   ,  1.   ,  1.   ,
         0.   ,  1.   , -0.715],
       [ 1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         1.   ,  0.   ,  1.   ,  0.   ,  0.   ,  1.   ,  1.   ,  3.   ,
         4.   ,  4.   , -0.631]])

In [18]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [19]:
print(logistic_mod.intercept_)
print(logistic_mod.coef_)

[-0.629]
[[-3.547e-02 -8.908e-01  2.169e-01 -4.294e-02  1.238e-01  2.242e-01
   2.916e-02 -2.381e-01 -1.333e-01 -5.106e-01 -6.282e-01 -3.413e-04
  -1.096e+00  4.679e-01  1.124e-01 -8.779e-02  8.404e-01 -1.120e-01
   5.960e-01]]


In [20]:
probabilities = logistic_mod.predict_proba(X_test1)
print(probabilities[:15,:])

[[0.818 0.182]
 [0.539 0.461]
 [0.648 0.352]
 [0.733 0.267]
 [0.896 0.104]
 [0.96  0.04 ]
 [0.924 0.076]
 [0.911 0.089]
 [0.929 0.071]
 [0.736 0.264]
 [0.976 0.024]
 [0.419 0.581]
 [0.773 0.227]
 [0.627 0.373]
 [0.807 0.193]]


In [21]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.5)
print(np.array(scores[:500]))

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0
 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0
 0 1 1 1 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1
 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1
 1 0 1 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0
 0 0 0 1 1 1 1 1 0 0 0 1 

In [22]:
aw_test = pd.read_csv('aw_test.csv')
print(aw_test.shape)
aw_test.head()

(500, 23)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,18988,NaN,Courtney,A,Baker,NaN,8727 Buena Vista Ave.,NaN,Fremont,California,...,1/5/1945,Bachelors,Management,F,S,0,2,0,5,86931
1,29135,NaN,Adam,C,Allen,NaN,3491 Cook Street,NaN,Haney,British Columbia,...,10/4/1964,Bachelors,Skilled Manual,M,M,1,2,2,4,100125
2,12156,NaN,Bonnie,NaN,Raji,NaN,359 Pleasant Hill Rd,NaN,Burbank,California,...,1/12/1934,Graduate Degree,Management,F,M,1,2,0,4,103985
3,13749,NaN,Julio,C,Alonso,NaN,8945 Euclid Ave.,NaN,Burlingame,California,...,9/22/1958,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161
4,27780,NaN,Christy,A,Andersen,NaN,"42, boulevard Tremblay",NaN,Dunkerque,Nord,...,3/19/1965,High School,Manual,F,M,1,1,2,2,21876


In [23]:
categorical_columns

['Education', 'Gender', 'MaritalStatus']

In [24]:
temp

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [25]:
FeaturesT = encode_string(aw_test['Occupation'])
for col in categorical_columns:
    tempT = encode_string(aw_test[col])
    FeaturesT = np.concatenate([FeaturesT, tempT], axis = 1)

print(FeaturesT.shape)
print(FeaturesT[:20, :])   

(500, 14)
[[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.]]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [26]:
FeaturesT = np.concatenate([FeaturesT, np.array(aw_test[['HomeOwnerFlag','NumberCarsOwned', 
                            'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome']])], axis = 1)
print(FeaturesT.shape)
print(FeaturesT[:2, :]) 

(500, 19)
[[0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 2.000e+00 0.000e+00 5.000e+00 8.693e+04]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 0.000e+00
  1.000e+00 2.000e+00 2.000e+00 4.000e+00 1.001e+05]]


In [27]:
indxT = range(FeaturesT.shape[0])
indxT = ms.train_test_split(indxT, test_size=0, shuffle=False)
X_test = FeaturesT[indxT[0],:]

#y_test = np.ravel(labelsT[indxT[0]])
X_test[:19, :19]

array([[0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 2.000e+00, 0.000e+00, 5.000e+00,
        8.693e+04],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        1.000e+00, 0.000e+00, 1.000e+00, 2.000e+00, 2.000e+00, 4.000e+00,
        1.001e+05],
       [0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 1.000e+00, 2.000e+00, 0.000e+00, 4.000e+00,
        1.040e+05],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        1.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 4.000e+00,
        1.272e+05],
       [0.000e+00, 0.000e+00, 1.

In [28]:
scaler = preprocessing.StandardScaler().fit(X_test[:,18:])
X_test[:,18:] = scaler.transform(X_test[:,18:])
X_test[:19,:19]

array([[ 0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  1.   ,  0.   ,  2.   ,
         0.   ,  5.   ,  0.127],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  1.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  1.   ,  1.   ,  0.   ,  1.   ,  2.   ,
         2.   ,  4.   ,  0.452],
       [ 0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  0.   ,  1.   ,  0.   ,  1.   ,  0.   ,  1.   ,  2.   ,
         0.   ,  4.   ,  0.547],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  1.   ,  1.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  4.   ,  1.118],
       [ 0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,
         0.   ,  0.   ,  1.   ,  0.   ,  1.   ,  0.   ,  1.   ,  1.   ,
         2.   ,  2.   , -1.476],
       [ 1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         1.   ,  0.   ,  1.   ,  0.   ,  1.

In [29]:
X_test.shape

(500, 19)

In [30]:
X_train

array([[ 1.   ,  0.   ,  0.   , ...,  0.   ,  1.   , -0.715],
       [ 1.   ,  0.   ,  0.   , ...,  4.   ,  4.   , -0.631],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  5.   ,  0.762],
       ...,
       [ 1.   ,  0.   ,  0.   , ...,  0.   ,  4.   , -0.096],
       [ 0.   ,  1.   ,  0.   , ...,  4.   ,  4.   ,  1.132],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   , -0.743]])

In [31]:
X_test

array([[ 0.   ,  1.   ,  0.   , ...,  0.   ,  5.   ,  0.127],
       [ 0.   ,  0.   ,  0.   , ...,  2.   ,  4.   ,  0.452],
       [ 0.   ,  1.   ,  0.   , ...,  0.   ,  4.   ,  0.547],
       ...,
       [ 0.   ,  1.   ,  0.   , ...,  3.   ,  3.   ,  1.387],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  2.   ,  0.485],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  4.   , -1.545]])

In [32]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [33]:
#X_test.shape

(500, 19)

In [34]:
probabilities = logistic_mod.predict_proba(X_test)
print(probabilities[:,:])

[[0.892 0.108]
 [0.485 0.515]
 [0.968 0.032]
 [0.776 0.224]
 [0.855 0.145]
 [0.959 0.041]
 [0.215 0.785]
 [0.186 0.814]
 [0.185 0.815]
 [0.855 0.145]
 [0.553 0.447]
 [0.952 0.048]
 [0.928 0.072]
 [0.969 0.031]
 [0.807 0.193]
 [0.899 0.101]
 [0.809 0.191]
 [0.693 0.307]
 [0.071 0.929]
 [0.534 0.466]
 [0.944 0.056]
 [0.843 0.157]
 [0.109 0.891]
 [0.734 0.266]
 [0.94  0.06 ]
 [0.718 0.282]
 [0.724 0.276]
 [0.087 0.913]
 [0.445 0.555]
 [0.922 0.078]
 [0.25  0.75 ]
 [0.568 0.432]
 [0.977 0.023]
 [0.786 0.214]
 [0.282 0.718]
 [0.157 0.843]
 [0.136 0.864]
 [0.641 0.359]
 [0.871 0.129]
 [0.717 0.283]
 [0.653 0.347]
 [0.534 0.466]
 [0.845 0.155]
 [0.917 0.083]
 [0.838 0.162]
 [0.893 0.107]
 [0.861 0.139]
 [0.104 0.896]
 [0.466 0.534]
 [0.879 0.121]
 [0.916 0.084]
 [0.367 0.633]
 [0.766 0.234]
 [0.955 0.045]
 [0.964 0.036]
 [0.849 0.151]
 [0.892 0.108]
 [0.568 0.432]
 [0.968 0.032]
 [0.683 0.317]
 [0.934 0.066]
 [0.976 0.024]
 [0.761 0.239]
 [0.228 0.772]
 [0.69  0.31 ]
 [0.137 0.863]
 [0.389 0.

In [35]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.5)
print(scores.size)
print(np.array(scores[:]))

500
[0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1 1
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 0 0 0
 0 1 1 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1
 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0
 1 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 1 1 0 0 1
 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1
 0 1 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1
 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 1 0 1
 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1 1
 0 1 1 0 0 1 0 0 0 1 

In [36]:
df = pd.DataFrame(scores)
df.head()

,0
0,0
1,1
2,0
3,0
4,0


In [37]:
df.to_csv('aw_test_prediction.csv')